In [1]:
# 이진분류
# positive class (1) : 좀더 중요한(심각한)클래스 'M'
# negativce class(0) : 'B'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
path='https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'

In [7]:
df=pd.read_csv(path,header=None)

In [9]:
X = df.iloc[:,2:] #2번째 열부터 마지막까지
y = df.iloc[:, 1] #1번째 열

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
pd.DataFrame(y).value_counts()

0    357
1    212
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
pipe_line = make_pipeline(StandardScaler(), PCA(n_components=2),
LogisticRegression(random_state=1))

In [13]:
pipe_line.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])

In [16]:
p=pipe_line.predict(X_test)

In [15]:
pipe_line.score(X_test,y_test)

0.956140350877193

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,p) #실제값, 예측값 순
# 4명은 실제 맞는데 아니라고 한거
# 1명은 아닌데 맞다고 오진단한거

array([[71,  1],
       [ 4, 38]], dtype=int64)

### 모델튜닝 : 모델의 하이퍼 파라미터 조정하는 법
#### 모델튜닝=모델선택
인수 선택및 조정 방법  
여러개 시도후 좋은 모델 선택  
모델 튜닝에 사용하는 데이터를 검증용 데이터라고 한다.  
#### 교차검증 
#### 5-fold  : 4/5는 훈련, 1/5는 검증 이런식으로 검증데이터를 바꿈 즉 5번 반복해서 할 수 있는 방법
#### C, gamma를 알아내기 위해서는 교차검증등의 방법을 활용해서 구한다

In [22]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(estimator=pipe_line,X=X_train,y=y_train,cv=10,n_jobs=-1)

In [23]:
scores

array([0.93478261, 0.93478261, 0.95652174, 0.95652174, 0.93478261,
       0.95555556, 0.97777778, 0.93333333, 0.95555556, 0.95555556])

In [24]:
np.mean(scores)

0.9495169082125603

In [25]:
# 하이퍼파라미터 선택방법
#결정트리 사용
from sklearn.tree import DecisionTreeClassifier

In [29]:
best_score=0
#반복문으로 작성
for k in range(1,11):
    scores=cross_val_score(estimator=DecisionTreeClassifier(max_depth=k,random_state=1),X=X_train,y=y_train,cv=5,n_jobs=-1)
    print(k,np.mean(scores))
    if np.mean(scores)>best_score:
        best_score=np.mean(scores)
        best_parameter={'max_depth':k}

1 0.8813186813186814
2 0.9318681318681319
3 0.9296703296703297
4 0.9406593406593406
5 0.9384615384615385
6 0.9362637362637363
7 0.9406593406593406
8 0.9406593406593406
9 0.9406593406593406
10 0.9406593406593406


In [30]:
best_score

0.9406593406593406

In [31]:
best_parameter

{'max_depth': 4}

In [34]:
best_tree=DecisionTreeClassifier(**best_parameter)

In [36]:
best_tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=4)

In [37]:
best_tree.score(X_test,y_test)

0.9385964912280702

### 그리드 서치 방법으로 위의 내용을 대신 처리할 수 있음 

In [38]:
from sklearn.model_selection import GridSearchCV

In [39]:
my_grid={'max_depth':range(1,11)}

In [40]:
gscv=GridSearchCV(estimator=DecisionTreeClassifier(random_state=1), param_grid=my_grid,scoring='accuracy',cv=5,n_jobs=-1)

In [41]:
gscv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': range(1, 11)}, scoring='accuracy')

In [42]:
gscv.best_score_

0.9406593406593406

In [43]:
gscv.best_params_

{'max_depth': 4}

In [45]:
# 6*6=36개 
from sklearn.svm import SVC
pipe_svc = make_pipeline(StandardScaler(), SVC(kernel='rbf', random_state=1))
p_list = [0.001,0.01,0.1,1.0,10.0,100.0]
param_grid = {'svc__C':p_list, 'svc__gamma':p_list}
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)
print(gs.best_params_) #{'svc__C': 10.0, 'svc__gamma': 0.01}
print(gs.score(X_test, y_test)) #0.9739

{'svc__C': 10.0, 'svc__gamma': 0.01}
0.9736842105263158


### 과제 4 : y는 yes,no 로 나옴
앞 정보를 가지고 가입할지 안할지 분류 모델을 만들기

In [47]:
# 앞으로는 딥러닝과 신경망 부분 학습
# 텐서플로우 설치해서 오기